#### Análise Exploratória da Base de Dados

In [1]:
import pandas as pd
from scipy.stats import shapiro

In [2]:
df = pd.read_csv("../Data/dataset_traduzido.csv")

In [3]:
df.head()

,Temperatura,Umidade,PM2.5,PM10,NO2,SO2,CO,Proximidade_Areas_Industriais,Densidade_Populacional,Qualidade_Ar
0,29.8,59.1,5.2,17.9,18.9,9.2,1.72,6.3,319,Moderate
1,28.3,75.6,2.3,12.2,30.8,9.7,1.64,6.0,611,Moderate
2,23.1,74.7,26.7,33.8,24.4,12.6,1.63,5.2,619,Moderate
3,27.1,39.1,6.1,6.3,13.5,5.3,1.15,11.1,551,Good
4,26.5,70.7,6.9,16.0,21.9,5.6,1.01,12.7,303,Good


In [4]:
print(f"Número de linhas: {df.shape[0]}; Número de colunas: {df.shape[1]}")

Número de linhas: 5000; Número de colunas: 10


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Temperatura                    5000 non-null   float64
 1   Umidade                        5000 non-null   float64
 2   PM2.5                          5000 non-null   float64
 3   PM10                           5000 non-null   float64
 4   NO2                            5000 non-null   float64
 5   SO2                            5000 non-null   float64
 6   CO                             5000 non-null   float64
 7   Proximidade_Areas_Industriais  5000 non-null   float64
 8   Densidade_Populacional         5000 non-null   int64  
 9   Qualidade_Ar                   5000 non-null   object 
dtypes: float64(8), int64(1), object(1)
memory usage: 390.8+ KB


In [6]:
df.isnull().sum()

Temperatura                      0
Umidade                          0
PM2.5                            0
PM10                             0
NO2                              0
SO2                              0
CO                               0
Proximidade_Areas_Industriais    0
Densidade_Populacional           0
Qualidade_Ar                     0
dtype: int64

In [7]:
contagem_nulos = df.isnull().sum()
if contagem_nulos.sum() == 0:
    print("Não há valores nulos no DataFrame.")
else:
    print("Colunas com valores nulos:")
    print(contagem_nulos[contagem_nulos > 0])

Não há valores nulos no DataFrame.


#### Estatística

In [8]:
df_copia = df.copy(deep=True)

In [9]:
df_copia.drop(columns=["Qualidade_Ar"], inplace=True, axis=1)

In [11]:
Q1 = df_copia.quantile(0.25)
Q3 = df_copia.quantile(0.75)
IQR = Q3 - Q1

descricao_transposta = df_copia.describe().T
estatisticas_adicionais = pd.DataFrame({
    'skew': df_copia.skew(),
    'kurtosis': df_copia.kurtosis(),
    'range': df_copia.max() - df_copia.min(),
    'valores_unicos': df_copia.nunique(),
    'outliers': ((df_copia < (Q1 - 1.5 * IQR)) | (df_copia > (Q3 + 1.5 * IQR))).sum()
})
descricao_aprimorada = pd.concat([descricao_transposta, estatisticas_adicionais], axis=1)

In [12]:
descricao_aprimorada

,count,mean,std,min,25%,50%,75%,max,skew,kurtosis,range,valores_unicos,outliers
Temperatura,5000.0,30.029020,6.720661,13.40,25.10,29.00,34.000,58.60,0.752187,0.513162,45.20,362,72
Umidade,5000.0,70.056120,15.863577,36.00,58.30,69.80,80.300,128.10,0.280528,-0.290316,92.10,723,19
PM2.5,5000.0,20.142140,24.554546,0.00,4.60,12.00,26.100,295.00,2.890910,13.033781,295.00,815,352
PM10,5000.0,30.218360,27.349199,-0.20,12.30,21.70,38.100,315.80,2.534815,10.273039,316.00,955,324
NO2,5000.0,26.412100,8.895356,7.40,20.10,25.30,31.900,64.90,0.638783,0.248461,57.50,445,73
SO2,5000.0,10.014820,6.750303,-6.20,5.10,8.00,13.725,44.90,1.166772,1.329247,51.10,348,124
CO,5000.0,1.500354,0.546027,0.65,1.03,1.41,1.840,3.72,0.879068,0.209653,3.07,265,45
Proximidade_Areas_Industriais,5000.0,8.425400,3.610944,2.50,5.40,7.90,11.100,25.80,0.469752,-0.233748,23.30,179,16
Densidade_Populacional,5000.0,497.423800,152.754084,188.00,381.00,494.00,600.000,957.00,0.204231,-0.473810,769.00,683,7


In [13]:
for coluna in df_copia.columns:
    if df_copia[coluna].notnull().sum() > 2: 
        stat, p = shapiro(df_copia[coluna])  
        print(f"\nColuna: {coluna}")
        print(f"Shapiro-Wilk Teste: Estatística={stat:.3f}, p={p:.3f}")
        if p > 0.05:
            print("Os dados seguem uma distribuição normal.")
        else:
            print("Os dados NÃO seguem uma distribuição normal.")
    else:
        print(f"\nColuna: {coluna}")
        print("A coluna possui poucos valores para o teste de normalidade.")


Coluna: Temperatura
Shapiro-Wilk Teste: Estatística=0.965, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: Umidade
Shapiro-Wilk Teste: Estatística=0.990, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: PM2.5
Shapiro-Wilk Teste: Estatística=0.715, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: PM10
Shapiro-Wilk Teste: Estatística=0.769, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: NO2
Shapiro-Wilk Teste: Estatística=0.972, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: SO2
Shapiro-Wilk Teste: Estatística=0.912, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: CO
Shapiro-Wilk Teste: Estatística=0.919, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: Proximidade_Areas_Industriais
Shapiro-Wilk Teste: Estatística=0.953, p=0.000
Os dados NÃO seguem uma distribuição normal.

Coluna: Densidade_Populacional
Shapiro-Wilk Teste: Estatística=0.990, p=0.000
Os dados NÃO seguem uma distribuição norm